In [1]:
from __init__ import *
from dataset import IdiomDataset
from collate import collate
from model import IdiomExtractor
from bert_embedder import BERTEmbedder
from hparams import HParams
from trainer import Trainer
from utils import *

SEED = 2
# set seeds to get reproducible results
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
# gpuda bazen randomluk olabiliyormuş onu kaldırmak için
torch.backends.cudnn.deterministic = True

# create bert
it_model_name = 'bert-base-multilingual-cased'
# output hidden states -> it helps to get hidden states from bert
it_config = BertConfig.from_pretrained(it_model_name, output_hidden_states=True)
it_tokenizer = BertTokenizer.from_pretrained(it_model_name)
# get bert weights
hf_it_model = BertModel.from_pretrained(it_model_name, config=it_config)


# Türkçe BERT
tr_model_name = "dbmdz/bert-base-turkish-128k-cased"
tr_config = BertConfig.from_pretrained(tr_model_name, output_hidden_states=True)
tr_tokenizer = BertTokenizer.from_pretrained(tr_model_name)
hf_tr_model = BertModel.from_pretrained(tr_model_name, config=tr_config)

# train, update or test mode selection
#mode = input("Do you want to train or test the model? (train, update, test): ").strip().lower()
mode = "train"
assert mode in ['train', 'update', 'test'], "Mode must be one of train, update, test"
# select the dataset
#dataset_selection = input("Select the dataset (ID10M, ITU, PARSEME, ALL_COMBINED): ").strip().upper()
dataset_selection = "ITU"
assert dataset_selection in ['ID10M', 'ITU', 'PARSEME', 'COMBINED'], "Dataset must be one of ID10M, ITU, PARSEME, COMBINED"

# check dataset path
tr_path = r"./src/checkpoints/tr/"
it_path = r"./src/checkpoints/it/"
os.makedirs(tr_path, exist_ok=True)
os.makedirs(it_path, exist_ok=True)

if mode in ["test","update"]:
    # list available checkpoints
    print("Available tr checkpoints:")
    checkpoints = os.listdir(tr_path)
    for i, checkpoint in enumerate(checkpoints):
        print(f"{i+1}. {checkpoint}")
    print("none")
    # load the model
    checkpoint = input("Enter the checkpoint (without .pt): ").strip()
    if checkpoint == "none":
        tr_path = None
    else:
        tr_path = tr_path + checkpoint + ".pt"
        assert os.path.exists(tr_path), "Model path does not exist"

    print("\n")

    print("Available it checkpoints:")
    checkpoints = os.listdir(it_path)
    for i, checkpoint in enumerate(checkpoints):
        print(f"{i+1}. {checkpoint}")
    print("none")
    # load the model
    checkpoint = input("Enter the checkpoint (without .pt): ").strip()
    if checkpoint == "none":
        it_path = None
    else:
        it_path = it_path + checkpoint + ".pt"
        assert os.path.exists(it_path), "Model path does not exist"

model_name = None
if mode in ["train", "update"]:
    #model_name = input("Enter the model name (without .pt): ").strip()
    model_name = "deneme"

elif mode == "test":
    model_name = checkpoint

# get stanza tagger for both languages
tagger_dict = initialize(use_gpu=True)

# get the path for the dataset
main_path = r"../resources/"+dataset_selection+"/"
train_file = main_path + "train.tsv"
dev_file = main_path + "dev.tsv"
test_file = main_path + "test.tsv"

labels_vocab = {"<pad>":0, "B-IDIOM":1, "I-IDIOM":2, "O":3}

# initialize the dataset
train_dataset, dev_dataset, test_dataset = None, None, None
if mode in ["train", "update"]:
    train_dataset = IdiomDataset(train_file, labels_vocab, tagger_dict)
    dev_dataset = IdiomDataset(dev_file, labels_vocab, tagger_dict)
    print(f"train sentences: {len(train_dataset)}")
    print(f"dev sentences: {len(dev_dataset)}")
    print("-" * 50 + "\n")
else:
    test_dataset = IdiomDataset(test_file, labels_vocab, tagger_dict) 
    print(f"test sentences: {len(test_dataset)}")
    print("-" * 50 + "\n")

#dataloader

if mode in ["train", "update"]:
    train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate)
    dev_dataloader = DataLoader(dev_dataset, batch_size=16, collate_fn=collate)
    print(f"length of train dataloader: {len(train_dataloader)}")
    print(f"length of dev dataloader: {len(dev_dataloader)}")
else:
    test_dataloader = DataLoader(test_dataset, batch_size=1, collate_fn=collate)
    print(f"length of test dataloader: {len(test_dataloader)}")


#instantiate the hyperparameters
params = HParams()

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#instantiate the model
it_model = IdiomExtractor(hf_it_model,
                    params).cuda()

it_model.freeze_bert()

tr_model = IdiomExtractor(hf_tr_model,
                    params).cuda()

tr_model.freeze_bert()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

it_embedder =  BERTEmbedder(hf_it_model, it_tokenizer, device)
tr_embedder =  BERTEmbedder(hf_tr_model, tr_tokenizer, device)


2025-05-10 12:15:48 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


Initializing Stanza pipelines...


2025-05-10 12:15:49 INFO: Downloaded file to C:\Users\krbyk\stanza_resources\resources.json
2025-05-10 12:15:50 INFO: Loading these models for language: tr (Turkish):
| Processor | Package       |
-----------------------------
| tokenize  | imst          |
| mwt       | imst          |
| lemma     | imst_nocharlm |

2025-05-10 12:15:50 INFO: Using device: cuda
2025-05-10 12:15:50 INFO: Loading: tokenize
2025-05-10 12:15:50 INFO: Loading: mwt
2025-05-10 12:15:50 INFO: Loading: lemma
2025-05-10 12:15:51 INFO: Done loading processors!
2025-05-10 12:15:51 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-05-10 12:15:51 INFO: Downloaded file to C:\Users\krbyk\stanza_resources\resources.json
2025-05-10 12:15:51 INFO: Loading these models for language: it (Italian):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| lemma     | combined_nocharlm |

2025-05-10 12:15:51 INFO: Using device: cuda
2025-05-10 12:15:51 INFO: Loading: tokenize
2025-05-10 12:15:51 INFO: Loading: mwt
2025-05-10 12:15:51 INFO: Loading: lemma
2025-05-10 12:15:52 INFO: Done loading processors!


Stanza pipelines initialized.
--------------------------------------------------

Reading dataset...



...

Data encoded.

--------------------------------------------------

Reading dataset...



15311it [00:00, 915191.51it/s]


Dataset read.

--------------------------------------------------

Encoding data...



100%|██████████| 1437/1437 [00:00<00:00, 48131.08it/s]


Data encoded.

--------------------------------------------------

train sentences: 11514
dev sentences: 1437
--------------------------------------------------

length of train dataloader: 720
length of dev dataloader: 90
BERT model parameters have been frozen.
BERT model parameters have been frozen.


In [2]:
example_sentence = next(iter(train_dataloader))

In [3]:
words, labels, langs = example_sentence

# embedding'i aynı size a getirmek için lazım
# labellar beraber padleniyor embeddingler ayrı
# yani en uzun tr cümle 14, it cümle 12 uzunluğundaysa
# it label 14, it embedding 12 boyutunda oluyor hata veriyor.
global_max = labels.size(1)

print(f"shape of words: {len(words)}, example: {words[0]}")
print(f"shape of labels: {len(labels)}, example: {labels[0]}")
print(f"shape of langs: {len(langs)}, example: {langs}")
print(global_max)

shape of words: 16, example: ['Zaman', 'kazanmak', 'için', 'yaptığın', 'entrikalar', 'seni', 'kurtarmayacak', ',', 'eninde', 'sonunda', 'yakalayacak', 'seni', 'polis', '!']
shape of labels: 16, example: tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], device='cuda:0')
shape of langs: 16, example: tensor([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1], device='cuda:0')
14


In [4]:
tr_indices = (langs == 0).nonzero(as_tuple=True)[0]
it_indices = (langs == 1).nonzero(as_tuple=True)[0]
print(f"tr_indices: {tr_indices}")
print(f"it_indices: {it_indices}")

tr_indices: tensor([ 0,  1,  5,  7, 10, 11, 12], device='cuda:0')
it_indices: tensor([ 2,  3,  4,  6,  8,  9, 13, 14, 15], device='cuda:0')


In [5]:
print(f"labels: {labels}")

labels: tensor([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0],
        [1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0],
        [2, 3, 1, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 1, 2, 2, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 1, 2, 3, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0],
        [3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0],
        [3, 3, 3, 1, 3, 2, 2, 2, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 2, 0, 0, 0]], device='cuda:0')


In [6]:
tr_words = [words[i] for i in tr_indices.cpu().numpy()]
tr_labels = labels[tr_indices] # 0 1 5 7 10 11 12. labellar tr dilinin labelları

print(f"length of tr_words: {len(tr_words)}, example: {tr_words[0]}")
print(f"shape of tr_labels: {tr_labels.shape}, example: {tr_labels}")

length of tr_words: 7, example: ['Zaman', 'kazanmak', 'için', 'yaptığın', 'entrikalar', 'seni', 'kurtarmayacak', ',', 'eninde', 'sonunda', 'yakalayacak', 'seni', 'polis', '!']
shape of tr_labels: torch.Size([7, 14]), example: tensor([[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0],
        [2, 3, 1, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 1, 2, 3, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0]], device='cuda:0')


In [7]:
# embed tr list of embeddings herbirisi 10,seq_length,768 -> [tr_batch_size, seq_length, 768]
tr_embedded = tr_embedder.embed_sentences(tr_words)
print(f"shape of embedded_tr: {len(tr_embedded)}, example: {tr_embedded[0].shape}")
print(f"shape of embedded_tr: {len(tr_embedded)}, example: {tr_embedded[1].shape}")

shape of embedded_tr: 7, example: torch.Size([14, 768])
shape of embedded_tr: 7, example: torch.Size([10, 768])


In [8]:
# tr embedded'i -1le padleyerek seq_lengthi eşitliyoruz. -> [tr_batch_size, max_seq_length, 768]
tr_embs = pad_sequence(tr_embedded, batch_first=True, padding_value=0).to(device)

# check embedding size to match the labels
if tr_embs.size(1) < global_max:
    # ekstra pad lazım
    pad_size = global_max - tr_embs.size(1)

    tr_embs = F.pad(tr_embs, (0, 0, 0, pad_size), "constant", 0)


print(f"shape of tr_embs: {tr_embs.shape}, example: {tr_embs[0]}")

shape of tr_embs: torch.Size([7, 14, 768]), example: tensor([[ 0.3040, -1.6375,  0.9913,  ...,  0.6738,  1.7467,  0.5538],
        [-0.4647,  0.5055, -0.9381,  ...,  3.0271,  3.8738, -0.3686],
        [ 3.6571, -1.3875, -0.9558,  ...,  1.5002,  2.6590,  1.7305],
        ...,
        [-2.0102, -1.6403, -0.8219,  ..., -1.0591,  1.9849, -4.5420],
        [-0.4976, -2.3514,  1.0389,  ...,  1.2063,  2.7370, -4.2313],
        [ 0.6042, -0.8540,  0.2265,  ..., -1.0613,  1.4576, -5.0178]],
       device='cuda:0')


In [9]:
it_words = [words[i] for i in it_indices.cpu().numpy()]
it_labels = labels[it_indices]

print(f"shape of it_labels: {it_labels.shape}, example: {it_labels}")
print(f"shape of it_words: {len(it_words)}, example: {it_words[0]}")

shape of it_labels: torch.Size([9, 14]), example: tensor([[1, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0],
        [3, 1, 2, 2, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0],
        [3, 3, 3, 1, 3, 2, 2, 2, 0, 0, 0, 0, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0],
        [3, 3, 3, 3, 3, 3, 3, 3, 1, 2, 2, 0, 0, 0]], device='cuda:0')
shape of it_words: 9, example: ['Prendi', 'con', 'le', 'pinze', 'quello', 'che', 'non', 'è', 'detto', 'da', 'esperti']


In [10]:
# embed tr list of embeddings herbirisi 10,seq_length,768 -> [tr_batch_size, seq_length, 768]
it_embedded = it_embedder.embed_sentences(it_words)
print(f"shape of embedded_it: {len(it_embedded)}, example: {it_embedded[0].shape}")
print(f"shape of embedded_it: {len(it_embedded)}, example: {it_embedded[1].shape}")

shape of embedded_it: 9, example: torch.Size([11, 768])
shape of embedded_it: 9, example: torch.Size([9, 768])


In [11]:
it_embs = pad_sequence(it_embedded, batch_first=True, padding_value=0).to(device)

# check embedding size to match the labels
if it_embs.size(1) < global_max:
    # ekstra pad lazım
    pad_size = global_max - it_embs.size(1)

    it_embs = F.pad(it_embs, (0, 0, 0, pad_size), "constant", 0)

print(f"shape of it_embs: {it_embs.shape}, example: {it_embs[0]}")

shape of it_embs: torch.Size([9, 14, 768]), example: tensor([[ 1.9155,  0.1461, -1.2802,  ...,  0.8907,  0.8552,  0.4204],
        [ 1.1491,  3.5726,  0.6087,  ..., -0.1244,  2.5063,  1.2722],
        [ 0.5323, -0.2845,  0.3650,  ..., -2.2384,  2.5075,  1.3367],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0')


In [12]:
tr_LL, _ = tr_model(tr_embs, tr_labels)
tr_NLL = -tr_LL

it_LL,_ = it_model(it_embs, it_labels)
it_NLL = -it_LL

print(f"tr_NLL: {tr_NLL}")
print(f"it_NLL: {it_NLL}")
loss = tr_NLL + it_NLL
print(f"loss: {loss}")

tr_optimizer = optim.Adam(tr_model.parameters(), lr=0.0001)
it_optimizer = optim.Adam(it_model.parameters(), lr=0.0001)

# Optimizer step
tr_optimizer.zero_grad()
it_optimizer.zero_grad()
loss.backward()
torch.nn.utils.clip_grad_norm_(tr_model.parameters(), 1)
torch.nn.utils.clip_grad_norm_(it_model.parameters(), 1)
tr_optimizer.step()
it_optimizer.step()


tr_NLL: 102.74681854248047
it_NLL: 157.33689880371094
loss: 260.0837097167969


In [13]:
# get one batch from dev set
eval_data = next(iter(dev_dataloader))
words, labels, langs = eval_data

# tr ve it nin indexlerini ayıkla
tr_indices = (langs == 0).nonzero(as_tuple=True)[0]
it_indices = (langs == 1).nonzero(as_tuple=True)[0]

batch_size, seq_len = labels.shape
device = labels.device
global_max = seq_len

hidden_size = tr_embedder.bert_model.config.hidden_size

# TR cümleleri ayıkla, embeddle ve petle
tr_words  = [words[i] for i in tr_indices.cpu().numpy()]
tr_labels = labels[tr_indices]
# türkçe cümle gelmiş mi kontrolü
if len(tr_words) > 0:
    tr_embedded = tr_embedder.embed_sentences(tr_words)
    tr_embs     = pad_sequence(tr_embedded, batch_first=True, padding_value=0).to(device)
    if tr_embs.size(1) < global_max:
        # ekstra pad lazım
        pad_size = global_max - tr_embs.size(1)
        tr_embs  = F.pad(tr_embs, (0, 0, 0, pad_size), "constant", 0)
else:
    # hiç TR cümlesi yoksa boş batch oluştur
    tr_embs = torch.zeros((0, global_max, hidden_size), device=device)

# IT cümleleri ayıkla, embeddle ve petle
it_words    = [words[i] for i in it_indices.cpu().numpy()]
it_labels   = labels[it_indices]
if len(it_words) > 0:
    it_embedded = it_embedder.embed_sentences(it_words)
    it_embs     = pad_sequence(it_embedded, batch_first=True, padding_value=0).to(device)
    if it_embs.size(1) < global_max:
        # ekstra pad lazım
        pad_size = global_max - it_embs.size(1)
        it_embs  = F.pad(it_embs, (0, 0, 0, pad_size), "constant", 0)
else:
    # hiç IT cümlesi yoksa boş batch oluştur
    it_embs = torch.zeros((0, global_max, hidden_size), device=device)

# take predictions
tr_decode = tr_model(tr_embs, None)
print(f"tr_decode: {tr_decode}")
it_decode = it_model(it_embs, None)

# alt kısım GPT bakmak lazım 

def decode_to_tensor(decode_out, seq_len, device):
    # 1) list of lists → list of 1D tensors
    token_tensors = [torch.tensor(seq, dtype=torch.long, device=device)
                     for seq in decode_out]
    # 2) hiç prediction yoksa boş tensor
    if not token_tensors:
        return torch.zeros((0, seq_len), dtype=torch.long, device=device)
    # 3) pad_sequence ile batch_first ve padding_value=-1
    padded = pad_sequence(token_tensors, batch_first=True, padding_value=-1)
    # 4) eğer hâlâ seq_len’den kısa ise sağa pad et
    if padded.size(1) < seq_len:
        pad_amt = seq_len - padded.size(1)
        padded = F.pad(padded, (0, pad_amt), value=-1)
    return padded

# kullanım:
tr_pred = decode_to_tensor(tr_decode, seq_len, device)
it_pred = decode_to_tensor(it_decode, seq_len, device)


# list of lists → tensor’a çevir
if len(tr_decode) > 0:
    tr_pred = torch.full((len(tr_decode), seq_len),
                         fill_value=-1, dtype=torch.long, device=device)
    for i, seq in enumerate(tr_decode):
        tr_pred[i, :len(seq)] = torch.tensor(seq, device=device)
else:
    tr_pred = torch.zeros((0, seq_len), dtype=torch.long, device=device)

if len(it_decode) > 0:
    it_pred = torch.full((len(it_decode), seq_len),
                         fill_value=-1, dtype=torch.long, device=device)
    for i, seq in enumerate(it_decode):
        it_pred[i, :len(seq)] = torch.tensor(seq, device=device)
else:
    it_pred = torch.zeros((0, seq_len), dtype=torch.long, device=device)

# 1) Tam batch için pred tensor’ü oluştur
all_pred = torch.full(
    (batch_size, seq_len),
    fill_value=-1,                # pad bölgelerde -1 kalsın
    dtype=torch.long,
    device=device
)

# 2) TR ve IT tahminlerini ilgili index’lere yerleştir
all_pred[tr_indices] = tr_pred
all_pred[it_indices] = it_pred

# 3) Sadece gerçek token’lar üzerinde doğru/yanlış kontrolü
valid_mask   = labels.ne(-1)        # True olan yerler gerçek token
correct_mask = (all_pred == labels) & valid_mask

# 4) İsterseniz token‐seviye accuracy
token_accuracy = correct_mask.sum().float() / valid_mask.sum().float()
print(f"Token‐seviye accuracy: {token_accuracy:.4f}")

# 5) Veya örnek‐seviye (sentence‐seviye) doğru karar:
#    her cümlenin tüm token’ları doğru mu?
sent_correct     = correct_mask.all(dim=1) & valid_mask.any(dim=1)
# valid_mask.any(dim=1) ile “tamamen pad’li” örnekleri atlıyoruz
sentence_accuracy = sent_correct.sum().float() / sent_correct.numel()
print(f"Sentence‐seviye accuracy: {sentence_accuracy:.4f}")

# all_pred şimdi (batch_size, seq_len) shape’inde, 
# her satır orijinal sırasıyla tahminleri içeriyor.


tr_decode: [[0, 1, 0, 0, 3, 3, 0, 0, 0, 1, 0, 2, 3, 2, 3, 2, 3, 3], [0, 0, 0, 0, 1, 0, 3, 2, 2, 2, 2, 3, 2, 3, 2, 3, 2, 0], [1, 3, 2, 0, 2, 0, 0, 3, 0, 2, 2, 3, 1, 2, 3, 0, 0, 0], [2, 2, 0, 3, 2, 3, 0, 1, 1, 0, 3, 1, 0, 0, 0, 0, 1, 0], [1, 0, 3, 0, 3, 2, 2, 3, 2, 0, 3, 3, 2, 0, 0, 1, 1, 2], [0, 3, 0, 3, 0, 0, 0, 0, 1, 2, 3, 1, 3, 2, 3, 0, 3, 0], [3, 0, 2, 0, 0, 0, 3, 3, 2, 3, 1, 1, 2, 2, 3, 0, 0, 3], [1, 1, 0, 1, 0, 1, 3, 3, 0, 2, 3, 0, 0, 1, 0, 2, 0, 2], [2, 2, 0, 3, 0, 0, 3, 0, 3, 2, 0, 3, 0, 3, 2, 3, 2, 0], [3, 0, 0, 0, 3, 2, 2, 3, 2, 3, 3, 0, 3, 2, 3, 2, 0, 0], [3, 1, 3, 2, 2, 3, 2, 1, 3, 0, 3, 0, 0, 1, 3, 0, 3, 2], [0, 0, 1, 3, 3, 2, 2, 1, 1, 0, 3, 0, 0, 2, 3, 0, 1, 1], [3, 3, 2, 3, 0, 3, 2, 0, 2, 0, 0, 0, 0, 1, 2, 3, 3, 3], [0, 0, 1, 2, 2, 1, 0, 2, 3, 0, 0, 3, 0, 3, 0, 1, 3, 3], [0, 2, 3, 0, 3, 3, 0, 2, 3, 0, 2, 0, 0, 0, 3, 3, 2, 0], [2, 0, 0, 3, 2, 2, 3, 1, 3, 2, 3, 3, 0, 0, 0, 3, 3, 0]]
Token‐seviye accuracy: 0.3403
Sentence‐seviye accuracy: 0.0000


In [14]:
trainer = Trainer(tr_model = tr_model, it_model = it_model,
                tr_optimizer = optim.Adam(tr_model.parameters(), lr=0.00001),
                it_optimizer = optim.Adam(it_model.parameters(), lr=0.00001),
                tr_embedder= tr_embedder,
                it_embedder= it_embedder,
                modelname = model_name,
                labels_vocab=labels_vocab)

trainer.train(train_dataloader, dev_dataloader, 30, patience=20)



Training...
 Epoch 001, patience: 20


Epoch 1: 100%|██████████| 720/720 [01:23<00:00,  8.63it/s]


[E:01] train tr_loss=30090.9453, it_loss=68.5010, total=30159.4473


Evaluating: 100%|██████████| 90/90 [00:05<00:00, 17.96it/s]




Full Accuracy: 0.8511, Full F1: 0.3065
              precision    recall  f1-score   support

           1     0.0000    0.0000    0.0000       319
           2     0.0000    0.0000    0.0000       319
           3     0.8849    1.0000    0.9389      4905

    accuracy                         0.8849      5543
   macro avg     0.2950    0.3333    0.3130      5543
weighted avg     0.7830    0.8849    0.8309      5543



TR Accuracy: 0.8849, TR F1: 0.3130
              precision    recall  f1-score   support

           1     0.0000    0.0000    0.0000       763
           2     0.0000    0.0000    0.0000      1303
           3     0.8511    1.0000    0.9196     11808

    accuracy                         0.8511     13874
   macro avg     0.2837    0.3333    0.3065     13874
weighted avg     0.7244    0.8511    0.7826     13874



IT Accuracy: 0.8286, IT F1: 0.3021
              precision    recall  f1-score   support

           1     0.0000    0.0000    0.0000       444
           2  

...

Epoch 30: 100%|██████████| 720/720 [01:26<00:00,  8.32it/s]


[E:30] train tr_loss=245.3492, it_loss=20.2987, total=265.6479


Evaluating: 100%|██████████| 90/90 [00:04<00:00, 18.26it/s]




Full Accuracy: 0.9381, Full F1: 0.8601
              precision    recall  f1-score   support

           1     0.7208    0.9467    0.8184       319
           2     0.6697    0.9279    0.7779       319
           3     0.9917    0.9466    0.9686      4905

    accuracy                         0.9455      5543
   macro avg     0.7940    0.9404    0.8550      5543
weighted avg     0.9575    0.9455    0.9490      5543



TR Accuracy: 0.9455, TR F1: 0.8550
              precision    recall  f1-score   support

           1     0.7567    0.8847    0.8157       763
           2     0.7483    0.8626    0.8014      1303
           3     0.9770    0.9499    0.9632     11808

    accuracy                         0.9381     13874
   macro avg     0.8274    0.8991    0.8601     13874
weighted avg     0.9434    0.9381    0.9399     13874



IT Accuracy: 0.9331, IT F1: 0.8611
              precision    recall  f1-score   support

           1     0.7886    0.8401    0.8135       444
           2  

([tensor(30159.4473, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(10690.7422, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(6952.6367, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(5071.7266, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(3896.5745, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(3082.0820, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(2477.2949, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(2048.9019, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(1725.6592, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(1496.7126, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(1306.1852, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(1134.0746, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(1008.0186, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(906.5610, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(793.6445, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(715.2738, device='cuda:0', grad_fn=<AddBackward0>),
  tensor(